---
title: Turbulent exchange of heat and water vapour
subject: Tutorial
subtitle: Notebooks to evaluate the relationships between canopy structure, turbulent transport of heat and water and surface temperature.
short_title: Sensible Heat Flux
authors:
  - name: Héctor Nieto
    affiliations:
      - Instituto de Ciencias Agrarias, ICA
      - CSIC
    orcid: 0000-0003-4250-6424
    email: hector.nieto@ica.csic.es
  - name: Benjamin Mary
    affiliations:
      - Insituto de Ciencias Agrarias
      - CSIC
    orcid: 0000-0001-7199-2885
  - name: Radoslaw Guzinski
    affiliations:
      - DHI
    orcid: 0000-0003-0044-6806
license: CC-BY-SA-4.0
keywords: TSEB, sensible heat, aerodynamic resistance, wind attenuation,
myst:
  enable_extensions: ["deflist", "attrs_block", "attrs_inline"]
---

# Summary
This interactive notebook has the objective of demonstrating the relationships between canopy structure, turbulent transport of heat and water and surface temperature.

For this purpose, we will use simulations based on energy combination models, both the "big  leaf" [Penman-Monteith](https://doi.org/10.1016/C2010-0-66393-0) model as well the dual source [Shuttleworth-Wallace](https://doi.org/10.1002/qj.49711146910) model.

# Instructions
Read carefully all the text and follow the instructions.

Once each section is read, run the jupyter code cell underneath (marked as `In []`) by clicking the icon `Run`, or pressing the keys SHIFT+ENTER of your keyboard. A graphical interface will then display, which allows you to interact with and perform the assigned tasks.

To start, please run the following cell to import all the packages required for this notebook. Once you run the cell below, an acknowledgement message, stating all libraries were correctly imported, should be printed on screen.

In [1]:
from ipywidgets import interactive, fixed
from IPython.display import display
import functions.tseb_and_resistances as fn

scikit-learn-intelex not available, skipping the use af Intel acceleration.
Try to consider installing Intel acceleration via `pip install scikit-learn-intelex`
Thanks! libraries imported
You can now continue with the following tasks


# Basic equations
As a reminder, the estimation of evapotranspiration (ET), or latent heat flux ($\lambda E$) in terms of energy, can be obtained from the energy balance equation:

$$\lambda E \approx R_n - G - H$$

In the previous exercise, we have already seen the theory and processes related to net radiation ($R_n$) and soil heat flux ($G$). Now, we will focus on the sensible heat flux ($H$) to finally get $\lambda E$ or ET.

The effective transport of sensible heat can be computed as:

$$H = \rho_{air} C_p \frac{T_0 - T_a}{r_a}$$

* [$\rho_{air}$](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/meteo_utils.py#L151) and [$C_p$](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/meteo_utils.py#L65) are the density of air and its specific heat respectively, and they can be assumed to be practically constant.
* $T_0$ (K) is the aerodynamic temperature.
  :::{attention}
  Do not confuse this temperature with the radiometric surface temperature that is estimated with thermal infrared sensors
  :::
* $T_a$ (K) is the air temperature measured at a height above ground $z_T$, and significantly above the canopy height ($h_c$)
* $r_a$ is the aerodynamic resistance to heat transport (s/m), a reciprocal of which is the aerodynamic conductance.
  :::{attention}
  Do not confuse this aerodynamic conductance with the leaf stomatal conductance
  :::

This fundamental equation can be interpreted in a way that the heat exchange between the land surface and the atmosphere ($H$) is driven by a gradient of temperatures between the surface and the nearest layers of air above the canopy. This transport or exchange of heat is modulated by the aerodynamic resistance, which includes a series of complex mechanical and convective processes that enhance or impede the transport of heat between the surface and the air:

$$r_{a}=\frac{\left[\ln\left(\frac{z_{T}-d}{z_{0H}}\right)+\Psi_{H}\right] \left[\ln\left(\frac{z_{U}-d}{z_{0M}}\right)+\Psi'_{M}\right]}{k^2\,U}$$

* $U$ is the wind speed (m/s) measured at a height above ground $z_U$, and significantly above the canopy height ($h_c$).
* $z_{T}$ is the height (m) at which air temperature is measured or modelled.
* $z_{U}$ is the height (m) at which wind speed is measured or modelled.
* $d$ and $z_{0M}$ are the zero-plane displacement height and surface roughness length for momentum transport, respectively. Usually these two variables can be estimated using a simplified formula based on canopy/obstacle height ($h_c$)

:::{math}
z_{0M} &= 0.125 h_c\\
d_{0} &= 2/3 h_c
:::

* $z_{0H}$ is the roughness length for heat transport. Likewise, its estimation is usually computed as a fraction of  $z_{0M}$ (e.g. $$z_{0H} = 0.1 z_{0M}$$ for dense vegetation)

* $k=0.41$ is von Kàrman constant.

* $\Psi'_{H}$ and $\Psi'_{M}$ are semi-empirical functions that modulate the aerodynamic resistance and the vertical wind attenuation as function of atmospheric stability.
  :::{seealso}
  :class:dropdown
  If you want to know more details about their calculation, you can check the [pyTSEB GitGub source code for $\Psi'_{H}$](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L179)  and [for $\Psi'_{M}$](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L244).
    :::
From the equation above we can arithmetically deduce that the aerodynamic resistance to heat and vapour transport decreases with wind speed and with the surface aerodynamic roughness.

In the following tasks, we are going to disentangle all those processes step by step, allowing us to better understand and estimate $r_a$.

# The wind profile
The wind speed profile with respect to the height above the ground can be estimated according to a logarithmic function. Ignoring in this first step the effects of convective turbulence, and thus considering a neutral atmospheric stability:

$$u\left(z\right) =\frac{u_*}{k}\left[\log\left(\frac{z - d}{z_{0M}}\right)\right]$$

:::{note}
This logarithmic wind attenuation is closely related to the aerodynamic resistance. 
:::

:::{seealso}
:class:dropdown
You can also check the [pyTSEB GitHub source code](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/wind_profile.py#L38).
:::

The wind attenuation will be stronger for a larger surface roughness, expressed by the variables $z_{0M}$ and $d_{0}$. 

$u_{*}$ is the friction velocity. Likewise, simplifying and neglecting the effects of atmospheric stability, $u_{*}$ is computed as:

$$u_* = \frac{k\,U} {\log\left(\frac{z_u - d_0}{z_{0M}}\right))}$$

:::{seealso}
:class:dropdown
You can also check the [pyTSEB GitHub source code](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L357).
:::

Therefore, based on wind speed measurement at a given height, we could extrapolate the wind profile to estimate the wind speed right above the canopy ($h_c$):

$$u_C =\frac{u_*}{k}\left[\log\left(\frac{h_c - d}{z_{0M}}\right)\right]$$

## The wind profile within the canopy
As the wind passes through the canopy, it is mainly attenuated by the presence of foliage. This attenuation within the canopy and towards the ground can have implications on the heat and vapour exchange between the canopy and the ground.

Usually this attenuation is formulated assuming that the canopy is horizontally and vertically homogeneous. However most canopies, and in particular woody vegetation, show a foliar density that varies with height: plants usually have a stem from which branches diverge, starting at a certain height above ground, resulting in canopies with a more or less irregular shape.

In the following interactive plot, we are going to simulate the wind attenuation above and below the canopy, and how this profile diverges according to the canopy structure. We are going to compare a profile for both a vertically heterogeneous (based on [Massman et al., 2017](https://doi.org/10.1139/cjfr-2016-0354) mode) and homogeneous canopy (based on [Goudriaan, 1977](https://library.wur.nl/WebQuery/wurpubs/70980) model). In addition, we are going to see the standard wind profile as estimated in the reference ET method proposed by FAO. For this task, we are keeping a constant wind speed ($U=5$ m/s) measured 10m above ground ($z_U=10$ m), and assuming a neutral atmospheric stability.

:::{seealso}
:class:dropdown
You can check the GitHub source code for both the [Goudriaan](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/wind_profile.py#L102) and for the [Massman](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/wind_profile.py#L169) models
:::

In order to characterize our canopy, we are going to need:

* Leaf Area Index ($LAI$) and canopy height ($h_c$), these two parameters are common for the homogeneous and heterogeneous canopy.

* The height above ground in which the foliage begins ($h_{bottom}$), expressed as the ratio to the canopy height
  :::{note}
  By default this value is set at 0.5, meaning that the foliage starts at a height of $0.5 h_c$ m.
  :::

* The height at which the maximum foliage density is located ($h_{max}$), expressed as a ratio of the canopy length (i.e the difference between $h_c$ and $h_{bottom}$).

In the graph below, the left-panel plot shows the relative foliage density of the simulated canopy. 

The right-panel plot shows the wind speed attenuation from 10m to the ground (0m) for both the vertical heterogeneous (solid blue line) and homogeneous (dashed blue line) cases. A blue star indicates the wind speed right above the canopy, and the dashed black line shows the standard profile for the reference ET crop in FAO method (a homogeneous grass layer with $h_c=0.12$ m and $LAI\approx3$).

In [2]:
w_wind = interactive(fn.wind_profile_heterogeneous, lai=fn.w_lai, zol=fixed(0.), h_c=fn.w_hc, 
                     hb_ratio=fn.w_hb_ratio, h_c_max=fn.w_h_c_max)
display(w_wind)

interactive(children=(FloatSlider(value=2.0, continuous_update=False, description='LAI (m²/m²)', max=4.0, tool…

* See how different the wind profile can be compared to the FAO reference crop profile. This is particularly evident when $h_c$ is significantly larger than 0.12 m. In addition, with higher canopies, there will be clearer differences between a profile for a vertically homogenous crop compared to a heterogenous one.

* As a rule of thumb, the stronger the attenuation is, the more turbulent the exchange of heat and vapour will be. This turbulence will produce more eddies and a more efficient transport of heat and vapour between the soil, the plant and the atmosphere, since eddies move the parcels of (warm and humid) air in contact with the land surface towards the atmosphere.

  ::::{seealso}
  :class:dropdown
  :::{iframe} https://youtu.be/5YwnY0wPphA
  :width: 100%
  This is a YouTube video showing a simulation of eddies while surface roughness in increased. In this case it corresponds to an airfoil
  :::
  ::::

* See also that with higher leaf density (expressed in terms of `LAI`), the wind attenuation within the canopy will be stronger. This results in distinct heat and water exchange between the canopy and the ground.

* Finally, see that the strongest attenuation within the canopy occurs around the height of maximum foliage density. This is due to the fact that the wind is encountering a larger amount of obstacles (leaves) and, thus, looses kinetic energy and, hence, speed. On the contrary, in areas closer to the ground, where the foliage density is minimal, the wind is less attenuated.

## Effect on the aerodynamic resistance

In this task, you will explore how the aerodynamic resistance varies with surface roughness (characterized by canopy height), wind speed and atmospheric stability. 

### The atmospheric stability and convective turbulence
So far, we saw that arithmetically the aerodynamic resistance to heat and vapour transport decreases with wind speed and roughness. In other words, the exchange of heat and vapour between the surface and the atmosphere becomes more efficient with stronger winds and rougher surfaces.

Now, we are going to evaluate that with more unstable atmospheres, due to convection, the turbulence is enhanced and, thus, the heat exchange will be more efficient. 

The atmospheric stability can be expressed in relative terms as a ratio between aerodynamic roughness length ($z_{0m}$) and the Monin-Obukhov length ($\xi=z_{0m}/L$), the latter of which also depends on the sensible heat flux ($H$).

:::{seealso}
:class:dropdown
You can also check the [pyTSEB GigHub source code for the Monin-Obukhov length](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L134)
:::

The more negative this $\xi$ coefficient is, the more unstable the atmosphere is and, thus, the more efficient the transport of heat and vapour would be due to the convection of warm air upwards the cooler atmosphere.

:::{note}
:class:dropdown
As a reminder, the density of warm air is less than that of cool air, resulting in a vertical (and turbulent) transport of air upwards. 
[See a video of this phenomenon](https://youtu.be/OM0l2YPVMf8)
:::

:::{important}
At daytime, the land surface is absorbing heat (mainly due to sun irradiance) and, therefore, unstable conditions usually occur ($\xi < 0$) during the daytime. By contrast, at nighttime, the land surface can be cooler than the air and, thus, thermal inversion phenomena and atmospheric stability can occur ($\xi > 0$). 
:::

The next interactive plot will let you evaluate the joint effect of atmospheric stability, wind speed and canopy height/roughness on the characterization of aerodynamic resistance.

In [ ]:
w_ra = interactive(fn.plot_aerodynamic_resistance, zol=fn.w_zol, h_c=fn.w_hc)
display(w_ra)

:::{note}
:class:dropdown
All parameters for all the plots used in this notebook are synchronized: any parameter change in one of the interactive plots is as well changed for the other plots, with all the graphs updated accordingly. This allows a quick intercomparison between graphs and simulations.
:::

* See how the aerodynamic resistance reduces exponentially with wind speed, but also with surface roughness/canopy height. This issue can have crucial implications when interpreting the gradient of temperature between the surface and the air. 

  :::{warning}
  The same temperature gradient can indicate different levels, or lack thereof, of water stress, depending on the surface roughness and the actual meteorological conditions.
  ::: 

* Now change the stability coefficient and see how $r_a$ changes. The aerodynamic resistance is reduced with more negative $xi$ values for a given wind speed and surface roughness. By contrast, with stable conditions ($xi$ > 0) the vertical transport of heat and vapour is reduced (larger values of $r_a$).

# The evapotranspiration and the sensible heat flux
In this second part of the exercise, we are going to integrate everything we have learnt so far to evaluate their joint effect on evapotranspiration and the land surface temperature. As mentioned before, we will use two popular energy combination models to perform our simulations.

The main advantage of [Penman-Monteith](https://doi.org/10.1016/C2010-0-66393-0) model is its relative simplicity (this is the model used to estimate FAO's reference ET), while its main disadvantage is that it is not fully able to partition fluxes between soil and canopy (e.g. separation of soil evaporation and canopy transpiration). This issue has important implications when modelling ET over sparse vegetation or row crops.

:::{figure} ./input/figures/penman_monteith_model.png
:alt:Penman Monteith model
:name: penman-monteith
Resistances layout in the [Penman-Monteith model](https://doi.org/10.1016/C2010-0-66393-0)
:::

:::{note}
:class:dropdown
The most well known application of Penman-Monteith model is the calculation of reference ET, e.g. [FAO 56](https://www.fao.org/3/x0490e/x0490e06.htm#chapter%202%20%20%20fao%20penman%20monteith%20equation) or [ASCE](https://doi.org/10.1061/9780784408056.fm), which estimates the evapotranspiration for a well watered crop used as reference. This reference crop for FAO 56 is defined as a perfectly homogeneous grass layer clipped at 12cm height.
:::

On the other hand, the [Shuttleworth-Wallace](https://doi.org/10.1002/qj.49711146910) model is more adapted to sparse vegetation conditions and is able to partition fluxes between soil and canopy, allowing thus the separation between soil evaporation and canopy transpiration. However, this model is more complex than Penman-Monteith and, thus, requires additional inputs.

:::{figure} ./input/figures/shuttleworth_wallace_model.png
:alt:Penman Monteith model
:name: penman-monteith
Resistances layout in the [Shuttleworth-Wallace model](https://doi.org/10.1002/qj.49711146910)
:::

Additional variables which are required by the Shuttleworth-Wallace model include the soil resistance of vapour transport ($R_{ss}$), which is related to the topsoil moisture content.

:::{tip} 
:class:dropdown
$R_{ss}$ values closer to 0 would indicate topsoil saturated or even flooded with water, whereas larger values will indicate a drier soil surface.
:::

## The effect of stomatal conductance
The leaf stomatal conductance or its reciprocal the leaf stomatal resistance ($g_s = 1 / r_{st}$) in an indicator of crop water stress. $g_s$ usually show values around a 0.4-0.5 mmol m$^{-2}$ s$^{-1}$ when the crop is well watered.

:::{note}
:class:dropdown
According to FAO guidelines, the stomatal resistance of the reference well-watered crop is 100 s/m, which is equivalent to a stomatal conductance of 0.415 mmol m$^{-2}$ s$^{-1}$ (at standard atmospheric conditions of 25ºC at sea level).
:::

When a soil water deficit at the root-zone occurs, the typical physiological response of plants is to close partially or totally their stomata, with the aim of avoiding significant losses of water. This closure has the consequence of a lower CO$_2$ assimilation through the stomata and, therefore, a reduction in the plants' productivity. This physiological stomatal closure is mathematically reflected in a decrease (increase) of stomata conductance (resistance).

## Flux interaction between ground and the canopy
Any exchange of heat and vapour between the ground, vegetation and the atmosphere is produced under a turbulent transport. Therefore, if the ground surface is dry and receives enough radiation it will heat up and, thus, will enhance the transport of dry and hot air towards the canopy. This would enhance the evaporative demand of the canopy (warming and drying the air around the leaves). Inversely, a dense and well irrigated canopy can provide cool and moist air towards the ground environment, decreasing its evaporative demand.

The following interactive plot will run simulations of [Penman-Monteith](https://doi.org/10.1016/C2010-0-66393-0) and [Shuttleworth-Wallace](https://doi.org/10.1002/qj.49711146910) models for a range of LAIs, given the following standard meteorological forcing:

* Solar irradiance of 300 W m$^{-2}$.
* Air temperature of 25ºC (298K).
* 50% relative humidity, which is equivalent to a vapour pressure deficit of $VPD\approx$ 15mb.
* Wind speed of 5 m s^{-1}$ measured at 10m above ground.

... and fixed surface albedo of 0.23 as defined for FAO's reference ET.

  :::{seealso}
  :class:dropdown
   You can check the PyTSEB GitHub source codes for the [Penman-Monteith model](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/energy_combination_ET.py#L30), for the [Shutlelworth-Wallace model](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/energy_combination_ET.py#L277), the [FAO56 reference ET](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/energy_combination_ET.py#L1068), and the [ASCE reference ET](https://github.com/hectornieto/pyTSEB/blob/382e4fc01e965143ebafdaefe5be9b45c737455a/pyTSEB/energy_combination_ET.py#L993)
  ::: 

The upper panel of the plot shows the simulated daily ET for the [Shuttleworth-Wallace](https://doi.org/10.1002/qj.49711146910) (blue line) and [Penman-Monteith](https://doi.org/10.1016/C2010-0-66393-0) (red line) models. In addition, it shows the FAO reference ET value with a black star.

The middle panel shows the ET partitioning between soil evaporation (yellow line) and canopy transpiration (green line) as simulated by Shuttleworth-Wallace.

(ET-simulations)=
The lower panel shows the [Shuttleworth-Wallace](https://doi.org/10.1002/qj.49711146910) simulated soil (yellow line), canopy (green) and aerodynamic temperatures (black line), expressed as their respective differences from the air temperature.


In [ ]:
w_et = interactive(fn.fluxes_and_resistances, r_ss=fn.w_r_ss, g_st=fn.w_g_st, h_c=fn.w_hc)
display(w_et)

* Observe how the graphs change with $R_{ss}$ (e.g. the topsoil moisture). See how the total ET in Shuttleworth-Wallace changes as opposed to the Penman-Monteith that can not directly consider the topsoil moisture. Observe how ET increases with LAI for the given parameters, since surface resistance decreases with LAI ($R_c = r_{st}/LAI$). 

  :::{warning}
  :class:dropdown
  Remember that $R_{ss}$ represents the topsoil moisture condition. Indeed, it is possible (more often than one could expect) to have a dry soil surface ($R_{ss} >> 1000$ s m$^{-1}$) while having a well-watered canopy ($g_{st} >> 0 mmol m$^{-2}$ s$^{-1}$), since plants extract water deeper in the soil, where water is still stored.
  :::

* See in the middle panel how transpiration dominates total ET at larger LAI values. In the previous exercise, we showed that, at larger LAI values, the canopy will intercept and absorb more energy, and thus will have a larger evaporative capacity.

* Look now at the ET values for $LAI\rightarrow0$ when $R_{ss}\rightarrow0$. Penman-Monteith, since it is a "big-leaf" model and does not directly incorporate the transport of water from the soil surface, is not able to simulate the soil surface evaporation. On the other hand, with $R_{ss}$ values above 5000 s/m both models yield very similar values of ET.

* Observe in the lower panel how temperatures decrease with LAI: a dense canopy dissipates the heat more efficiently than a less dense canopy, even under the same water stress conditions. 

* Check also how soil temperature increases with $R_{ss}$. 

  :::{tip}
  :class:dropdown
  You could even perceive how the leaf temperature changes slightly with $R_{ss}$, especially at lower LAI values. This is related to the flux interaction between ground and canopy that we mentioned before: a dry and warm (moist and cool) soil transports air towards the canopy, which can increase (decrease) the evaporative demand and, thus, decrease (increase) the canopy temperature.
   :::
  
* Keep a value of $R_{ss}=5000$ fixed (dry soil surface) and check now how ET changes with decreasing values of stomatal conductance, and thus simulating more severe crop water stress conditions. The ET for both Penman-Monteith and Shuttleworth-Wallace decreases and deviates further from FAO's reference ET value (optimal water conditions). In addition, the more stressed the plant (i.e. at lower $g_{st}$ values), the lower the transpiration, with no transpiration for $g_s=0$ mmol m$^{-2}$ s$^{-1}$. Finally, observe that the lower $g_{st}$, the hotter the leaf and aerodynamic temperatures.

  :::{tip}
  :class:dropdown
   You could even perceive how the soil temperature slightly changes with $g_{st}$, especially at larger LAI values. This is again related to the flux interaction between ground and canopy that we mentioned before: a dry and warm (moist and cool) canopy transports air towards the ground, which can increase (decrease) the evaporative demand and thus decrease (increase) the soil temperature.
  :::
  
* See now the effect of canopy height/roughness. Restore the values of stomatal conductance back to $g_s=0.41$ mmol m$^{-2}$ s$^{-1}$ and $R_{ss}=5000$. Increase now the canopy height, from an initial value of 0.12m, and observe how ET and temperature changes with canopy roughness for a given moisture status. 

# About the surface temperature
We already mentioned that the aerodynamic temperature ($T_0$), which is the temperature that holds valid the sensible heat flux equation, is different to the radiometric surface temperature (LST) estimated with Earth observation remote sensing. It is therefore important to be aware of such difference and to be able to correctly interpret LST when estimating sensible heat flux and/or ET.

:::{note}
:class:dropdown
Indeed, the different remote sensing ET models mainly differ on how they use the LST in order to retrieve $H$ and/or $\lambda E$.
:::

Firstly, since heat is exchanged less efficiently than momentum, we need to correctly define the aerodynamic roughness length for heat transport ($z_0H$), and, indeed, the relationship $z_{0H} \approx 0.1 z_{0M}$ might not be valid for all types of surfaces.

Secondly, the radiometric surface temperature can significantly change depending on the observation viewing angle. This is particularly relevant for Earth observation sensors with a wide field of view or wide swath, such as Sentinel-3 SLSTR, with view zenith angles ranging from nadir ($VZA = 0$º) up to VZAs of 50º or 60º. This is important since the sensor might see a different proportion of (cool) vegetation and (warm) soil. Indeed, we saw in the previous exercise that the canopy transmittance depends on the canopy density, the leaf angle distribution, and the incidence/illumination angle. This effect is perfectly reciprocal when we deal with the sensor geometry instead of the solar geometry.

The interactive plot below demonstrates this effect. You can define the sensor's view zenith angle as well as the dominant leaf inclination angle. According to the soil and canopy temperatures estimated in the [previous simulation](#ET-simulations), and after setting as well the soil and leaf emissivity, the graph will display the estimated LST for a range of LAIs.

:::{tip}
:class:dropdown
You can modify the simulation parameters [here](#ET-simulations) and the changes will be reflected in the simulated LST.
:::

The plot shows two different methods of estimating the radiometric land surface temperature:
* A detailed calculation based on the 4SAIL model that includes thermal emission and multiple thermal scattering between soil and vegetation.
  :::{seealso}
  :class:dropdown
   You can check the [Python version of 4SAIL model](https://github.com/hectornieto/pypro4sail/blob/f6825f5b1325259cb014dcb934ccff2a9442a3ae/pypro4sail/four_sail.py#L445)
  ::: 

* A simplified method which considers a weighted average based on the fraction of vegetation observed by the sensor, which is the one implemented in TSEB:
$$LST^4=f_c\left(\theta\right)T_{C}^4+\left[1-f_{c}\left(\theta\right)\right]T_{S}^4$$
  :::{seealso}
  :class:dropdown
  You can check the [pyTSEB GitHub source code](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/TSEB.py#L2644)

In [ ]:
w_fveg = interactive(fn.get_land_surface_temperature,
                     vza=fn.w_vza, leaf_angle=fn.w_leaf_angle, temperatures=fixed(w_et), 
                     e_v=fn.w_ev, e_s=fn.w_es)
display(w_fveg)

* Notice that with increasing VZA, the LST is decreasing, with this decrease being more significant at higher VZAs (VZA > 35º) and for intermediate values of LAI.

* There might be significant differences between the aerodynamic temperature and the LST, in particular for less developed and sparse canopies (LAI < 2).


# Conclusions
In this exercise, we have seen that exchange of heat and vapour between ground, vegetation and the atmosphere depend on:
1. The topsoil moisture content, which drives the soil evaporation.
2. The root-zone soil moisture content, which drives the canopy transpiration and its photosynthetic capacity.
3. The canopy structure, which drives how efficiently the canopy dissipates the heat and vapour towards the atmosphere.

The radiometric land surface temperature is a key factor in order to evaluate the crop water stress and crop evapotranspiration. 


:::{note}
Please feel free comment any thoughts.
:::